# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

<br>
$
p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T \\
A = ((1-\alpha)M + \frac{\alpha}{N}E) \\
Opakujte: \\
\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}
$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [2]:
import numpy as np

In [3]:
path_data = 'data/'
paths = [path_data + 'graph_' + str(num_graph) + '.txt' for num_graph in range(1,7)]

In [4]:
#load graphs
graphs = {}
uniq_val = {}
for path in paths:
    graphs[path] = {}
    uniq_val[path] = []
    with open(path,'r') as file:
        for line in file:
            parent, child = list(map(int,line.replace('\n','').split(',')))
            uniq_val[path].extend([child, parent])
            if parent in graphs[path].keys():
                graphs[path][parent].append(child)
            else:
                graphs[path][parent] = [child]
    uniq_val[path] = set(uniq_val[path])
    

In [5]:
M = {} #mat. přechodů s pravděpodobnostmi přechodu do jiného uzlu
E = {} #jednotková / poctem uzlu
A = {} #mat. sousednosti
alphas = {} # alphy = O,2 default, stránky bez výstupních uzlů = 1
alpha = 0.2
for graph in graphs:
    N = len(uniq_val[graph])
    M[graph] = np.zeros((N,N))
    A[graph] = np.zeros((N,N))
    alphas[graph] = np.ones(N) * alpha
    for parent in graphs[graph].keys():
        num_child = len(graphs[graph][parent])
        for child in graphs[graph][parent]:
            M[graph][parent-1,child-1] = 1 / num_child
            A[graph][parent-1,child-1] = 1
    E[graph] = np.ones((N,N)) / N
    alphas[graph][np.where([np.all(M[graph] == 0, axis=1)])[1]] = 1
    


In [6]:
num_iter = 100
p = {}
for graph in graphs:
    N = len(uniq_val[graph])
    p[graph] = np.ones((N,1))/N
    for _ in range(num_iter):
        alpha = alphas[graph][:,np.newaxis]
        a = (1-alpha) * M[graph] + alpha * E[graph]
        p[graph] = a.T @ p[graph]
    print('\033[1m' + graph + '\033[0m')
    print(p[graph],'\n')


data/graph_1.txt
[[0.0656044 ]
 [0.11808792]
 [0.16007474]
 [0.19366419]
 [0.22053575]
 [0.242033  ]] 

data/graph_2.txt
[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]] 

data/graph_3.txt
[[0.17857143]
 [0.32142857]
 [0.32142857]
 [0.17857143]] 

data/graph_4.txt
[[0.27257372]
 [0.15666713]
 [0.13837881]
 [0.10924643]
 [0.18531604]
 [0.06563464]
 [0.07218322]] 

data/graph_5.txt
[[0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00163995]
 [0.00185861]
 [0.00177115]
 [0.00182737]
 [0.00196794]
 [0.00196794]
 [0.00182737]
 [0.00177115]
 [0.00196794]
 [0.00196794]
 [0.00177115]
 [0.00169711]
 [0.00170178]
 [0.0019428 ]
 [0.00198014]
 [0.00169465]
 [0.00365343]
 [0.003554  ]
 [0.00169465]
 [0.00402315]
 [0.00182136]
 [0.00185236]
 [0.00187664]
 [0.00170178]
 [0.00170178]
 [0.00240599]
 [0.00197445]
 [0.00175037]
 [0.00246671]
 [0.00225534]
 [0.00253521]
 [0.00193072]
 [0.00178986]
 [0.00168645]
 [0.00169465]
 [0.004043  ]
 [0.00170178]
 [0.00169465]
 [0.00369723]
 [0.00445807]
 [0.00170178]


### Předpokládaný výstup

#### graph_1.xt
```python
array([[0.0656044 ],
       [0.11808792],
       [0.16007474],
       [0.19366419],
       [0.22053575],
       [0.242033  ]])
```
       
#### graph_2.txt
```python
array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])
```


#### graph_3.txt
```python
array([[0.17857143],
       [0.32142857],
       [0.32142857],
       [0.17857143]])
```


#### graph_4.txt
```python
array([[0.27257372],
       [0.15666713],
       [0.13837881],
       [0.10924643],
       [0.18531604],
       [0.06563464],
       [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [7]:
num_iter = 100
for graph in graphs:
    N = len(uniq_val[graph])
    a = np.ones((N,1))
    h = np.ones((N,1))
    for _ in range(num_iter):
        h_new = A[graph] @ a
        a = A[graph].T @ h
        h /= np.linalg.norm(h_new,1)
        a /= np.linalg.norm(a,1)
    print('\033[1m' + graph + '\033[0m')
    print('Authority:',a)
    print('Hub:',h,'\n')

data/graph_1.txt
Authority: [[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]] 

data/graph_2.txt
Authority: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]] 

data/graph_3.txt
Authority: [[0.16666667]
 [0.33333333]
 [0.33333333]
 [0.16666667]]
Hub: [[1.81477395e-23]
 [1.81477395e-23]
 [1.81477395e-23]
 [1.81477395e-23]] 

data/graph_4.txt
Authority: [[0.22222222]
 [0.16666667]
 [0.16666667]
 [0.11111111]
 [0.22222222]
 [0.05555556]
 [0.05555556]]
Hub: [[8.83264603e-51]
 [8.83264603e-51]
 [8.83264603e-51]
 [8.83264603e-51]
 [8.83264603e-51]
 [8.83264603e-51]
 [8.83264603e-51]] 

data/graph_5.txt
Authority: [[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00090744]
 [0.00272232]
 [0.00090744]
 [0.00090744]
 [0.00272232]
 [0.00272232]
 [0.000907

### Předpokládaný výstup

#### graph_1.xt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

In [1]:
import pandas as pd
import spacy
import warnings
from collections import defaultdict
from functools import reduce
warnings.filterwarnings('ignore')

In [2]:
def lemmatize_text(text):
    return " ".join([token.lemma_ for token in lemmatizer(text)])

In [3]:
df = pd.read_csv('articles.csv', usecols=["title", "text"])
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
df['title'] = df['title'].str.replace('[^\w\s]|','').str.replace('\s\s+',' ').str.lower()
df['title'] = df['title'].apply(lemmatize_text)

In [4]:
def mapping(item):
    idx,row = item
    words = row.split(" ")
    d = dict.fromkeys(set(words),0)
    for word in set(words):
        d[word] = ("D" + str(idx),words.count(word))
    return d

titles = df['title'].tolist()
documents = list(map(mapping, enumerate(titles)))


In [7]:
def reduc(d1, d2):
    uniq_keys = set(list(d1.keys()) + list(d2.keys()))
    dd = dict.fromkeys(uniq_keys,[])
    for d in [d1,d2]: 
        for key, value in d.items():
            dd[key].append(value)
    return dd

In [8]:
inverted_index = reduce(reduc, documents)
print(inverted_index)

KeyboardInterrupt: 

### Předpokládaný výstup

```python
print(inverted_index['be'])
print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.